In [0]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
import time
import os
import copy

In [0]:
basePath = '/content/'

modelSave = basePath+'weights.pth'
data_dir = basePath+'data'

num_classes = 2
batch_size = 8
num_epochs = 1
input_size = 224

In [0]:
# Detect if we have a GPU available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [18]:
'''import zipfile
import os
for file_name in os.listdir('/content/'):
  if file_name.endswith('.zip'):
    with zipfile.ZipFile(file_name,'r') as zip_dir:
      zip_dir.extractall(path='/content/')'''

"import zipfile\nimport os\nfor file_name in os.listdir('/content/'):\n  if file_name.endswith('.zip'):\n    with zipfile.ZipFile(file_name,'r') as zip_dir:\n      zip_dir.extractall(path='/content/')"

In [0]:
#torch.utils.model_zoo.load_url('https://download.pytorch.org/models/alexnet-owt-4df8aa71.pth', model_dir='/content/')

In [0]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs=25):
    since = time.time()

    val_acc_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    outputs = model(inputs)
                    loss = criterion(outputs,labels)

                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history

In [0]:
def set_parameter_requires_grad(model):
    for param in model.parameters():
        param.requires_grad = False

In [0]:
def initialize_model(num_classes):
    alexnet = models.alexnet(pretrained=True)
    set_parameter_requires_grad(alexnet)
    num_ftrs = alexnet.classifier[6].in_features
    alexnet.classifier[6] = nn.Linear(num_ftrs,num_classes)
    return alexnet

In [0]:
def getTrainDataLoaders():
    # Data augmentation and normalization for training
    # Just normalization for validation
    data_transforms = {
        'train': transforms.Compose([
            transforms.RandomResizedCrop(input_size),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
        'val': transforms.Compose([
            transforms.Resize(input_size),
            transforms.CenterCrop(input_size),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
    }

    print("Initializing Datasets and Dataloaders...")

    # Create training and validation datasets
    image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'val']}
    # Create training and validation dataloaders
    dataloaders_dict = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True, num_workers=4) for x in ['train', 'val']}

    return dataloaders_dict

In [0]:
def getUpdatablePara(model):
    params_to_update = model.parameters()
    print("Params to learn:")
    params_to_update = []
    for name,param in model.named_parameters():
        if  param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
    return params_to_update

# **Training**

In [0]:
alexnet = initialize_model(num_classes)
alexnet = alexnet.to(device)
#print(alexnet)

In [26]:
dataloaders = getTrainDataLoaders()

Initializing Datasets and Dataloaders...


In [27]:
params_to_update = getUpdatablePara(alexnet)

Params to learn:
	 classifier.6.weight
	 classifier.6.bias


In [28]:
# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(params_to_update, lr=0.001, momentum=0.9)

#loss function 
criterion = nn.CrossEntropyLoss()

alexnet, hist = train_model(alexnet, dataloaders, criterion, optimizer_ft, num_epochs=num_epochs)

torch.save(alexnet.state_dict(), modelSave)

Epoch 0/0
----------
train Loss: 1.0516 Acc: 0.0000
val Loss: 0.2301 Acc: 1.0000

Training complete in 0m 1s
Best val Acc: 1.000000


# **Testing**

In [29]:
'''alexnet = initialize_model(num_classes)

alexnet.load_state_dict(torch.load(modelSave))
alexnet.eval()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
alexnet = alexnet.to(device)
'''

'alexnet = initialize_model(num_classes)\n\nalexnet.load_state_dict(torch.load(modelSave))\nalexnet.eval()\ndevice = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")\nalexnet = alexnet.to(device)\n'